In [1]:
import pandas as pd
import numpy as np
import os
import re

In [2]:
with open("all companies.txt", "r") as f:
    all_companies = [str(line.strip()) for line in f]

all_companies

['AAA',
 'AAM',
 'AAT',
 'ABR',
 'ABS',
 'ABT',
 'ACB',
 'ACC',
 'ACG',
 'ACL',
 'ADG',
 'ADP',
 'ADS',
 'AGG',
 'AGR',
 'AMD',
 'ANV',
 'APG',
 'APH',
 'ASG',
 'ASM',
 'ASP',
 'AST',
 'BAF',
 'BBC',
 'BCE',
 'BCG',
 'BCM',
 'BFC',
 'BHN',
 'BIC',
 'BID',
 'BKG',
 'BMC',
 'BMI',
 'BMP',
 'BRC',
 'BSI',
 'BSR',
 'BTP',
 'BTT',
 'BVH',
 'BWE',
 'C32',
 'C47',
 'CCC',
 'CCI',
 'CCL',
 'CDC',
 'CHP',
 'CIG',
 'CII',
 'CKG',
 'CLC',
 'CLL',
 'CLW',
 'CMG',
 'CMV',
 'CMX',
 'CNG',
 'COM',
 'CRC',
 'CRE',
 'CSM',
 'CSV',
 'CTD',
 'CTF',
 'CTG',
 'CTI',
 'CTR',
 'CTS',
 'CVT',
 'D2D',
 'DAH',
 'DAT',
 'DBC',
 'DBD',
 'DBT',
 'DC4',
 'DCL',
 'DCM',
 'DGC',
 'DGW',
 'DHA',
 'DHC',
 'DHG',
 'DHM',
 'DIG',
 'DLG',
 'DMC',
 'DPG',
 'DPM',
 'DPR',
 'DQC',
 'DRC',
 'DRH',
 'DRL',
 'DSC',
 'DSE',
 'DSN',
 'DTA',
 'DTL',
 'DTT',
 'DVP',
 'DXG',
 'DXS',
 'DXV',
 'EIB',
 'ELC',
 'EVE',
 'EVF',
 'EVG',
 'FCM',
 'FCN',
 'FDC',
 'FIR',
 'FIT',
 'FLC',
 'FMC',
 'FPT',
 'FRT',
 'FTS',
 'GAB',
 'GAS',
 'GDT',


In [3]:
def format_data(stock_data_file, report_data_file_1, report_data_file_2):
    #P
    stock_data = pd.read_csv(f'stock data/{stock_data_file}')

    # Chuyển cột 'date' thành datetime
    stock_data['Date'] = pd.to_datetime(stock_data['Date'], utc=True)

    # Tính quý tương ứng
    stock_data['quarter'] = stock_data['Date'].dt.to_period('Q')

    # Tính trung bình giá đóng cửa theo quý
    quarterly_avg = stock_data.groupby('quarter')['Close'].mean().reset_index()

    # Tạo danh sách quý từ 2005Q1 đến 2024Q4
    all_quarters = pd.period_range('2005Q1', '2024Q4', freq='Q')
    all_quarters_df = pd.DataFrame({'quarter': all_quarters})

    # Merge với dữ liệu thực tế, đảm bảo đủ các quý
    merged = pd.merge(all_quarters_df, quarterly_avg, on='quarter', how='left')

    # Đổi tên cột 'close' thành 'P'
    merged.rename(columns={'Close': 'P'}, inplace=True)
    
    # Đổi định dạng cột 'quarter'
    merged['quarter'] = merged['quarter'].astype(str)  # Chuyển cột 'quarter' thành chuỗi
    merged['quarter'] = merged['quarter'].apply(convert_quarter)  # Áp dụng hàm convert_quarter

    # Chọn cả cột 'quarter' và 'P' vào một DataFrame
    merged_quarterly = merged[['quarter', 'P']]

    # Chuyển DataFrame thành Series
    P = merged_quarterly.set_index('quarter')['P']

    # D/E
    report_data_1 = pd.read_csv(f'report 1/{report_data_file_1}', index_col=0)
    
    # Đổi tên cột
    report_data_1.columns = [convert_quarter(c) for c in report_data_1.columns]

    # Lấy feature thứ 65 và 95 (index 64 và 94)
    de_numerator = report_data_1.iloc[65].copy()
    de_denominator = report_data_1.iloc[95].copy()

    de_numerator_clean = clean_numeric(de_numerator)
    de_denominator_clean = clean_numeric(de_denominator)

    # Đồng bộ index và tính D/E
    common_quarters = de_numerator_clean.index.intersection(de_denominator_clean.index)
    de_ratio = de_numerator_clean[common_quarters] / de_denominator_clean[common_quarters]

    # Tạo bảng kết quả
    DE = de_ratio.rename('D/E')
    DE.index.name = 'quarter'

    report_data_2 = pd.read_csv(f'report 2/{report_data_file_2}', index_col=0)

    report_data_2.columns = [convert_quarter(c) for c in report_data_2.columns]

    line_21_fpt_2 = report_data_2.iloc[21]

    line_21_fpt_2_clean = clean_numeric(line_21_fpt_2)

    EPS = line_21_fpt_2_clean[common_quarters]

    EPS.index.name = 'quarter'

    EPS.name = 'EPS'

    common_quarters_2 = EPS.index.intersection(P.index)

    PE = P[common_quarters_2] / EPS[common_quarters_2]

    PE.name = 'P/E'

    de_size = report_data_1.iloc[63].copy()
    de_size_clean = clean_numeric(de_size)
    SIZE = np.log(de_size_clean).rename('SIZE')
    SIZE.index.name = 'quarter'

    result = pd.concat([P, EPS, DE, PE, SIZE], axis=1)

    result = result[~np.isinf(result['P/E'])]

    return result
    
# Loại bỏ dấu phẩy và chuyển về số
def clean_numeric(series):
    return pd.to_numeric(series.astype(str).str.replace(',', '', regex=False), errors='coerce')

# Chuẩn hóa tên cột từ "Quý ? năm ?" → "YYYYQ#"
def convert_quarter(col):
    match = re.match(r'Quý (\d+) năm (\d{4})', col)
    if match:
        q, y = match.groups()
        return f'{y}Q{q}'
    return col

In [4]:
for company_code in all_companies:
    try:
        stock_data_file = f"{company_code}_stock_data.csv"
        report_data_file_1 = f"Tổng hợp báo cáo {company_code}.csv"
        report_data_file_2 = f"Tổng hợp báo cáo {company_code} 2.csv"
    
        # Gọi format_data và lưu kết quả vào file CSV
        result = format_data(stock_data_file, report_data_file_1, report_data_file_2)
        result.to_csv(f"pre-processing/{company_code}_data.csv")
    except Exception as e:
        print(e)

C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\3866778413.py:9: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  stock_data['quarter'] = stock_data['Date'].dt.to_period('Q')
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\3866778413.py:9: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  stock_data['quarter'] = stock_data['Date'].dt.to_period('Q')
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\3866778413.py:9: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  stock_data['quarter'] = stock_data['Date'].dt.to_period('Q')
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\3866778413.py:9: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  stock_data['quarter'] = stock_data['Date'].dt.to_period('Q')
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\3866778413.py:9: UserWarning: Converting to PeriodArray/Index 

[Errno 2] No such file or directory: 'stock data/ADS_stock_data.csv'
[Errno 2] No such file or directory: 'stock data/AGR_stock_data.csv'


C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\3866778413.py:9: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  stock_data['quarter'] = stock_data['Date'].dt.to_period('Q')
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\3866778413.py:9: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  stock_data['quarter'] = stock_data['Date'].dt.to_period('Q')
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\3866778413.py:9: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  stock_data['quarter'] = stock_data['Date'].dt.to_period('Q')
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\3866778413.py:9: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  stock_data['quarter'] = stock_data['Date'].dt.to_period('Q')
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\3866778413.py:9: UserWarning: Converting to PeriodArray/Index 

[Errno 2] No such file or directory: 'stock data/AST_stock_data.csv'


C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\3866778413.py:9: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  stock_data['quarter'] = stock_data['Date'].dt.to_period('Q')
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\3866778413.py:9: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  stock_data['quarter'] = stock_data['Date'].dt.to_period('Q')
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\3866778413.py:9: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  stock_data['quarter'] = stock_data['Date'].dt.to_period('Q')
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\3866778413.py:9: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  stock_data['quarter'] = stock_data['Date'].dt.to_period('Q')
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\3866778413.py:9: UserWarning: Converting to PeriodArray/Index 

...

C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\3866778413.py:9: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  stock_data['quarter'] = stock_data['Date'].dt.to_period('Q')
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\3866778413.py:9: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  stock_data['quarter'] = stock_data['Date'].dt.to_period('Q')
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\3866778413.py:9: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  stock_data['quarter'] = stock_data['Date'].dt.to_period('Q')
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\3866778413.py:9: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  stock_data['quarter'] = stock_data['Date'].dt.to_period('Q')
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\3866778413.py:9: UserWarning: Converting to PeriodArray/Index 

[Errno 2] No such file or directory: 'stock data/VIB_stock_data.csv'


C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\3866778413.py:9: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  stock_data['quarter'] = stock_data['Date'].dt.to_period('Q')
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\3866778413.py:9: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  stock_data['quarter'] = stock_data['Date'].dt.to_period('Q')
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\3866778413.py:9: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  stock_data['quarter'] = stock_data['Date'].dt.to_period('Q')
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\3866778413.py:9: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  stock_data['quarter'] = stock_data['Date'].dt.to_period('Q')
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\3866778413.py:9: UserWarning: Converting to PeriodArray/Index 

[Errno 2] No such file or directory: 'stock data/VSI_stock_data.csv'
[Errno 2] No such file or directory: 'stock data/VTP_stock_data.csv'


C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\3866778413.py:9: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  stock_data['quarter'] = stock_data['Date'].dt.to_period('Q')
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\3866778413.py:9: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  stock_data['quarter'] = stock_data['Date'].dt.to_period('Q')
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\3866778413.py:9: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  stock_data['quarter'] = stock_data['Date'].dt.to_period('Q')


In [5]:
# Khởi tạo list chứa dữ liệu từng công ty
all_data = []

# Đọc từng file và gắn mã công ty
for ticker in all_companies:
    file_path = f"pre-processing/{ticker}_data.csv"
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)

        # Thêm mã công ty
        df["company_id"] = ticker

        # Chuyển cột thời gian nếu có
        if "quarter" in df.columns:
            df["quarter"] = pd.to_datetime(df["quarter"])

        all_data.append(df)
    else:
        print(f"⚠️ File không tồn tại: {file_path}")

# Gộp toàn bộ dữ liệu
df_all = pd.concat(all_data, ignore_index=True)

# Sắp xếp theo thời gian và công ty
df_all = df_all.sort_values(["company_id", "quarter"])

# Tạo biến mục tiêu P(t+1)
df_all["P_t_plus_1"] = df_all.groupby("company_id")["P"].shift(-1)

# Xoá dòng thiếu dữ liệu đầu vào
df_model = df_all.dropna(subset=["P_t_plus_1", "P/E", "EPS", "D/E", "SIZE"])

df_model.set_index(['company_id', 'quarter'], inplace=True)

df_model.to_csv('data.csv')


C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\3719418896.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["quarter"] = pd.to_datetime(df["quarter"])
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\3719418896.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["quarter"] = pd.to_datetime(df["quarter"])
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\3719418896.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["quarter"] = pd.to_datetime(df["quarter"])
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\3719418896.py:15: UserWarning: Could not infer for

⚠️ File không tồn tại: pre-processing/ADS_data.csv
⚠️ File không tồn tại: pre-processing/AGR_data.csv
⚠️ File không tồn tại: pre-processing/AST_data.csv
⚠️ File không tồn tại: pre-processing/BIC_data.csv
⚠️ File không tồn tại: pre-processing/BMI_data.csv
⚠️ File không tồn tại: pre-processing/BVH_data.csv
⚠️ File không tồn tại: pre-processing/CCC_data.csv


C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\3719418896.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["quarter"] = pd.to_datetime(df["quarter"])
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\3719418896.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["quarter"] = pd.to_datetime(df["quarter"])
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\3719418896.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["quarter"] = pd.to_datetime(df["quarter"])
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\3719418896.py:15: UserWarning: Could not infer for

⚠️ File không tồn tại: pre-processing/CTR_data.csv
⚠️ File không tồn tại: pre-processing/D2D_data.csv
⚠️ File không tồn tại: pre-processing/DHG_data.csv
⚠️ File không tồn tại: pre-processing/DPM_data.csv
⚠️ File không tồn tại: pre-processing/DSE_data.csv
⚠️ File không tồn tại: pre-processing/EVE_data.csv


C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\3719418896.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["quarter"] = pd.to_datetime(df["quarter"])
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\3719418896.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["quarter"] = pd.to_datetime(df["quarter"])
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\3719418896.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["quarter"] = pd.to_datetime(df["quarter"])
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\3719418896.py:15: UserWarning: Could not infer for

...

C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\3719418896.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["quarter"] = pd.to_datetime(df["quarter"])
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\3719418896.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["quarter"] = pd.to_datetime(df["quarter"])
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\3719418896.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["quarter"] = pd.to_datetime(df["quarter"])
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\3719418896.py:15: UserWarning: Could not infer for

⚠️ File không tồn tại: pre-processing/SMA_data.csv
⚠️ File không tồn tại: pre-processing/SRC_data.csv
⚠️ File không tồn tại: pre-processing/SSI_data.csv
⚠️ File không tồn tại: pre-processing/SZC_data.csv
⚠️ File không tồn tại: pre-processing/TBC_data.csv
⚠️ File không tồn tại: pre-processing/TCL_data.csv
⚠️ File không tồn tại: pre-processing/TN1_data.csv


C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\3719418896.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["quarter"] = pd.to_datetime(df["quarter"])
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\3719418896.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["quarter"] = pd.to_datetime(df["quarter"])
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\3719418896.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["quarter"] = pd.to_datetime(df["quarter"])
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\3719418896.py:15: UserWarning: Could not infer for

⚠️ File không tồn tại: pre-processing/TV2_data.csv
⚠️ File không tồn tại: pre-processing/VCF_data.csv
⚠️ File không tồn tại: pre-processing/VIB_data.csv
⚠️ File không tồn tại: pre-processing/VSI_data.csv
⚠️ File không tồn tại: pre-processing/VTP_data.csv


C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\3719418896.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["quarter"] = pd.to_datetime(df["quarter"])
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\3719418896.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["quarter"] = pd.to_datetime(df["quarter"])
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\3719418896.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["quarter"] = pd.to_datetime(df["quarter"])
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\3719418896.py:15: UserWarning: Could not infer for

In [7]:
# Khởi tạo list chứa dữ liệu từng công ty
all_data = []

# Đọc từng file và gắn mã công ty
for ticker in all_companies:
    file_path = f"pre-processing/{ticker}_data.csv"
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)

        # Thêm mã công ty
        df["company_id"] = ticker

        # Chuyển cột thời gian nếu có
        if "quarter" in df.columns:
            df["quarter"] = pd.to_datetime(df["quarter"])

        all_data.append(df)
    else:
        print(f"⚠️ File không tồn tại: {file_path}")

# Gộp toàn bộ dữ liệu
df_all = pd.concat(all_data, ignore_index=True)

# Sắp xếp theo thời gian và công ty
df_all = df_all.sort_values(["company_id", "quarter"])

df_all = df_all.drop('P/E', axis=1)

# Xoá dòng thiếu dữ liệu đầu vào
df_all = df_all.dropna(subset=['P', "EPS", "D/E", "SIZE"])

df_all.set_index(['company_id', 'quarter'], inplace=True)

df_all.to_csv('data_train.csv')


C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\2084060499.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["quarter"] = pd.to_datetime(df["quarter"])
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\2084060499.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["quarter"] = pd.to_datetime(df["quarter"])
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\2084060499.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["quarter"] = pd.to_datetime(df["quarter"])
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\2084060499.py:15: UserWarning: Could not infer for

⚠️ File không tồn tại: pre-processing/ADS_data.csv
⚠️ File không tồn tại: pre-processing/AGR_data.csv
⚠️ File không tồn tại: pre-processing/AST_data.csv
⚠️ File không tồn tại: pre-processing/BIC_data.csv
⚠️ File không tồn tại: pre-processing/BMI_data.csv
⚠️ File không tồn tại: pre-processing/BVH_data.csv
⚠️ File không tồn tại: pre-processing/CCC_data.csv
⚠️ File không tồn tại: pre-processing/CTR_data.csv
⚠️ File không tồn tại: pre-processing/D2D_data.csv
⚠️ File không tồn tại: pre-processing/DHG_data.csv
⚠️ File không tồn tại: pre-processing/DPM_data.csv


C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\2084060499.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["quarter"] = pd.to_datetime(df["quarter"])
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\2084060499.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["quarter"] = pd.to_datetime(df["quarter"])
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\2084060499.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["quarter"] = pd.to_datetime(df["quarter"])
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\2084060499.py:15: UserWarning: Could not infer for

⚠️ File không tồn tại: pre-processing/DSE_data.csv
⚠️ File không tồn tại: pre-processing/EVE_data.csv
⚠️ File không tồn tại: pre-processing/GEE_data.csv
⚠️ File không tồn tại: pre-processing/GMH_data.csv
⚠️ File không tồn tại: pre-processing/HOT_data.csv
⚠️ File không tồn tại: pre-processing/HRC_data.csv
⚠️ File không tồn tại: pre-processing/HT1_data.csv
⚠️ File không tồn tại: pre-processing/HU1_data.csv
⚠️ File không tồn tại: pre-processing/HU3_data.csv


C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\2084060499.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["quarter"] = pd.to_datetime(df["quarter"])
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\2084060499.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["quarter"] = pd.to_datetime(df["quarter"])
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\2084060499.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["quarter"] = pd.to_datetime(df["quarter"])
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\2084060499.py:15: UserWarning: Could not infer for

⚠️ File không tồn tại: pre-processing/KDC_data.csv
⚠️ File không tồn tại: pre-processing/L10_data.csv
⚠️ File không tồn tại: pre-processing/LCM_data.csv
⚠️ File không tồn tại: pre-processing/LGC_data.csv
⚠️ File không tồn tại: pre-processing/LM8_data.csv
⚠️ File không tồn tại: pre-processing/MCG_data.csv
⚠️ File không tồn tại: pre-processing/MIG_data.csv
⚠️ File không tồn tại: pre-processing/NAV_data.csv
⚠️ File không tồn tại: pre-processing/NCT_data.csv
⚠️ File không tồn tại: pre-processing/NNC_data.csv
⚠️ File không tồn tại: pre-processing/NO1_data.csv
⚠️ File không tồn tại: pre-processing/NT2_data.csv
⚠️ File không tồn tại: pre-processing/PGI_data.csv
⚠️ File không tồn tại: pre-processing/PVT_data.csv
⚠️ File không tồn tại: pre-processing/QNP_data.csv
⚠️ File không tồn tại: pre-processing/S4A_data.csv
⚠️ File không tồn tại: pre-processing/SBA_data.csv
⚠️ File không tồn tại: pre-processing/SC5_data.csv


C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\2084060499.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["quarter"] = pd.to_datetime(df["quarter"])
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\2084060499.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["quarter"] = pd.to_datetime(df["quarter"])
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\2084060499.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["quarter"] = pd.to_datetime(df["quarter"])
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\2084060499.py:15: UserWarning: Could not infer for

⚠️ File không tồn tại: pre-processing/SMA_data.csv
⚠️ File không tồn tại: pre-processing/SRC_data.csv
⚠️ File không tồn tại: pre-processing/SSI_data.csv
⚠️ File không tồn tại: pre-processing/SZC_data.csv
⚠️ File không tồn tại: pre-processing/TBC_data.csv
⚠️ File không tồn tại: pre-processing/TCL_data.csv
⚠️ File không tồn tại: pre-processing/TN1_data.csv
⚠️ File không tồn tại: pre-processing/TV2_data.csv
⚠️ File không tồn tại: pre-processing/VCF_data.csv
⚠️ File không tồn tại: pre-processing/VIB_data.csv


C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\2084060499.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["quarter"] = pd.to_datetime(df["quarter"])
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\2084060499.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["quarter"] = pd.to_datetime(df["quarter"])
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\2084060499.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["quarter"] = pd.to_datetime(df["quarter"])
C:\Users\hype1\AppData\Local\Temp\ipykernel_42892\2084060499.py:15: UserWarning: Could not infer for

⚠️ File không tồn tại: pre-processing/VSI_data.csv
⚠️ File không tồn tại: pre-processing/VTP_data.csv
